In [1]:
from torch.autograd import Variable
import torch
import pickle
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import evaluation_vectorization as ev
import random
import config

sys.path.append(os.path.join(os.getcwd(), 'Aging_Model'))
import FigureConfig as FC

# Device

In [2]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda:0')
device = 'cpu'
device

'cpu'

# Dataset

In [3]:
if config.current_dataset is not None:
    ds = config.datasets[config.current_dataset]
else:
    ds = config.datasets[0]

ds

'VertebralColumn_c3'

In [4]:
datapath = os.path.join(config.path, f'Dataset_{ds}.p')
with open(datapath, 'rb') as f:
    dataset = pickle.load(f)
X = dataset['X'].float()
y = dataset['y']
E, N_features, N_class = X.shape[0], X.shape[1], torch.max(
    torch.unique(y)).item()+1
X.shape, y.shape, E, N_features, N_class

(torch.Size([310, 6]), torch.Size([310]), 310, 6, 3)

## Pseudo-electrical Signal

In [5]:
# normalization
X = X - torch.min(X, axis=0)[0]
X = X / torch.max(X, axis=0)[0]
torch.min(X), torch.max(X)

(tensor(0.), tensor(1.))

In [6]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

# generate tensordataset
dataset = TensorDataset(X.to(device), y.to(device))

# split
train_rate = 0.6
valid_rate = 0.2
test_rate = 0.2
E_train = min(8096, int(E*train_rate))
E_valid = min(2048, int(E*valid_rate))
E_test = min(2048, int(E*test_rate))


random.seed(config.data_split_seed);
np.random.seed(config.data_split_seed);
torch.manual_seed(config.data_split_seed);

train_data, rest_data = random_split(dataset, [E_train, E-E_train])
valid_data, rest_data = random_split(rest_data, [E_valid, E-E_train-E_valid])
test_data, rest_data = random_split(rest_data, [E_test, E-E_train-E_valid-E_test])

len(train_data), len(valid_data), len(test_data)

(186, 62, 62)

In [7]:
# batch
train_loader = DataLoader(train_data, batch_size=len(train_data))
valid_loader = DataLoader(valid_data, batch_size=len(valid_data))
test_loader = DataLoader(test_data, batch_size=len(test_data))

# Evaluation

## Load Models

In [8]:
accs = []
losses = []

for seed in range(10):
    with open(f'./result/{ds}_AAPNN_{seed}.p', 'rb') as f:
        AAPNN = pickle.load(f)
    AAPNN.to(device)
    
    random.seed(0);
    np.random.seed(0);
    torch.manual_seed(0);
    
    mean_acc_AAPNN, std_acc_AAPNN, mean_maa_AAPNN, std_maa_AAPNN, loss = ev.Evaluation(AAPNN, valid_loader, config.M_test, int(config.M_test/10), config.K_test*10, device, T=10)
    accs.append(np.mean(mean_acc_AAPNN))
    losses.append(loss)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
best_seed = np.argmin(losses)
best_seed

2

In [10]:
losses[best_seed]

0.1847878396511078

In [11]:
with open(f'./result/{ds}_AAPNN_{best_seed}.p', 'rb') as f:
    AAPNN = pickle.load(f)
AAPNN.to(device)

Sequential(
  (0): PNNLayer()
  (1): PNNLayer()
)

In [12]:
mean_acc_AAPNN, std_acc_AAPNN, mean_maa_AAPNN, std_maa_AAPNN, loss = ev.Evaluation(AAPNN, test_loader, config.M_test, int(config.M_test/10), config.K_test*10, device, T=10)

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
loss

0.3431703746318817

In [14]:
np.savetxt(f'./result/figures/{ds}_AAPNN_ACC_{best_seed}_{int(np.mean(mean_acc_AAPNN)*1000)}.txt', np.vstack((mean_acc_AAPNN, std_acc_AAPNN)))
np.savetxt(f'./result/figures/{ds}_AAPNN_MAA_{best_seed}_{int(np.mean(mean_maa_AAPNN)*1000)}.txt', np.vstack((mean_maa_AAPNN, std_maa_AAPNN)))

In [15]:
if losses[best_seed]==0:
    if loss==0:
        overfitting = 1
    else:
        overfitting = 10000
else:
    overfitting = loss/losses[best_seed]
np.savetxt(f'./result/figures/{ds}_AAPNN_loss.txt', np.array([losses[best_seed], loss, overfitting]), fmt='%.10f')